In [36]:
import pandas as pd
import numpy as np

import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [37]:
train = pd.read_csv("click_train_ready_for_model.csv")
test = pd.read_csv("click_test_ready_for_model.csv")

In [38]:
result = pd.DataFrame(columns=["session_id","is_click"])

In [39]:
result["session_id"] = test["session_id"]

In [40]:
result["is_click"] = "1"

In [41]:
result.head()

,session_id,is_click
0,411705,1
1,208263,1
2,239450,1
3,547761,1
4,574275,1


In [62]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(my_pipeline, X, y, scoring='accuracy')
print(scores)

TypeError: get_params() missing 1 required positional argument: 'self'

In [42]:
#Define target and ID columns:
target = 'is_click'
IDcol = ['session_id']
from sklearn import cross_validation, metrics

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])

    #Perform cross-validation:
    cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=20, scoring='accuracy')
    cv_score = np.sqrt(np.abs(cv_score))
    
    #Print model report:
    print("\nModel Report")
    print("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

In [43]:
from sklearn.linear_model import LinearRegression
predictors = [x for x in train.columns if x not in [target]+IDcol]
print("predictors : ",predictors)
# print predictors
alg1 = LinearRegression(normalize=True)

predictors :  ['age_level', 'campaign_id', 'city_development_index', 'gender', 'product', 'product_category_1', 'product_category_2', 'user_depth', 'user_group_id', 'user_id', 'webpage_id']


In [44]:
X_train = train[predictors]

In [45]:
X_train.head()

,age_level,campaign_id,city_development_index,gender,product,product_category_1,product_category_2,user_depth,user_group_id,user_id,webpage_id
0,4.0,359520,3.0,0.0,2.0,4,0.0,3.0,10.0,858557,13787
1,2.0,105960,2.0,0.0,2.0,5,0.0,2.0,8.0,243253,11085
2,2.0,359520,2.0,0.0,2.0,4,0.0,2.0,8.0,243253,13787
3,3.0,359520,2.0,1.0,8.0,3,0.0,3.0,3.0,1097446,13787
4,2.0,405490,2.0,1.0,2.0,3,0.0,3.0,2.0,663656,60305


In [46]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463291 entries, 0 to 463290
Data columns (total 11 columns):
age_level                 463291 non-null float64
campaign_id               463291 non-null int64
city_development_index    463291 non-null float64
gender                    463291 non-null float64
product                   463291 non-null float64
product_category_1        463291 non-null int64
product_category_2        463291 non-null float64
user_depth                463291 non-null float64
user_group_id             463291 non-null float64
user_id                   463291 non-null int64
webpage_id                463291 non-null int64
dtypes: float64(7), int64(4)
memory usage: 38.9 MB


In [47]:
y_train = pd.DataFrame(train.is_click)

In [48]:
y_train.shape

(463291, 1)

In [49]:
X_test = test[predictors]

In [50]:
X_test.shape

(107137, 11)

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [52]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_train, y_train)))

Accuracy of logistic regression classifier on test set: 0.93


In [54]:
y_pred = logreg.predict(X_test)

In [55]:
y_pred.shape

(107137,)

In [56]:
result = pd.DataFrame(columns=["session_id","is_click"])
result["session_id"] = test.session_id
result["is_click"] = y_pred
result.to_csv("res2_logistic.csv",index=False)

In [57]:
res1=pd.read_csv("res2_logistic.csv")

In [58]:
res1.head()

,session_id,is_click
0,411705,0.0
1,208263,0.0
2,239450,0.0
3,547761,0.0
4,574275,0.0
